<a href="https://colab.research.google.com/github/bhargav23/OpenCV/blob/main/OpenCV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
imgpath =  "/content/drive/MyDrive/Data/AnnotationImages/"

In [ ]:
annotationpath = "/content/drive/MyDrive/Data/csv/"

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import cv2
from ast import literal_eval
import matplotlib
import matplotlib.pyplot as plt
from zipfile import ZipFile

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Data/csv/merged237.csv')

In [ ]:
ls

drive/  sample_data/


In [ ]:
dir_list = os.listdir(annotationpath)
for file in dir_list:
    print(annotationpath+file)

/content/drive/MyDrive/Data/csv/000001-000100_csv.csv
/content/drive/MyDrive/Data/csv/000200-000300_csv.csv
/content/drive/MyDrive/Data/csv/000501-000520_csv.csv
/content/drive/MyDrive/Data/csv/000401-000500_csv.csv
/content/drive/MyDrive/Data/csv/000200-000220_csv.csv
/content/drive/MyDrive/Data/csv/merged.csv
/content/drive/MyDrive/Data/csv/merged.gsheet
/content/drive/MyDrive/Data/csv/merged237.csv


In [ ]:
df1.head()

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,image000001.jpg,11432,{},1,0,"{""name"":""rect"",""x"":1,""y"":34,""width"":219,""heigh...","{""Entity"":""Bus"",""Door Location"":""NA"",""Vehicle ..."
1,image000002.jpg,72688,{},1,0,"{""name"":""rect"",""x"":373,""y"":263,""width"":104,""he...","{""Entity"":""Bus"",""Door Location"":""NA"",""Vehicle ..."
2,image000003.jpg,82261,{},3,0,"{""name"":""rect"",""x"":74,""y"":52,""width"":618,""heig...","{""Entity"":""Bus"",""Door Location"":""NA"",""Vehicle ..."
3,image000003.jpg,82261,{},3,1,"{""name"":""rect"",""x"":465,""y"":151,""width"":36,""hei...","{""Entity"":""Door"",""Door Location"":""Front Door"",..."
4,image000003.jpg,82261,{},3,2,"{""name"":""rect"",""x"":638,""y"":180,""width"":14,""hei...","{""Entity"":""Door"",""Door Location"":""Rear Door"",""..."


In [ ]:
df1['region_shape_attributes'] = df1.region_shape_attributes.apply(eval)

In [ ]:
df1['region_attributes'] = df1.region_attributes.apply(eval)

In [ ]:
df1.region_shape_attributes.apply(dict)
df1.region_attributes.apply(dict)

0       {'Entity': 'Bus', 'Door Location': 'NA', 'Vehi...
1       {'Entity': 'Bus', 'Door Location': 'NA', 'Vehi...
2       {'Entity': 'Bus', 'Door Location': 'NA', 'Vehi...
3       {'Entity': 'Door', 'Door Location': 'Front Doo...
4       {'Entity': 'Door', 'Door Location': 'Rear Door...
                              ...                        
1994    {'Entity': 'Door', 'Door Location': 'Front Doo...
1995    {'Entity': 'Bus', 'Door Location': 'NA', 'Vehi...
1996    {'Entity': 'Door', 'Door Location': 'Rear Door...
1997    {'Entity': 'Bus', 'Door Location': 'NA', 'Vehi...
1998    {'Entity': 'Route', 'Door Location': 'NA', 'Ve...
Name: region_attributes, Length: 1999, dtype: object

In [ ]:
#df1 = pd.concat([df1.drop(['region_shape_attributes'], axis=1), df1['region_shape_attributes'].apply(pd.Series)], axis=1)
#df1 = pd.concat([df1.drop(['region_attributes'], axis=1), df1['region_attributes'].apply(pd.Series)], axis=1)

In [ ]:
df1.head()

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,image000001.jpg,11432,{},1,0,"{'name': 'rect', 'x': 1, 'y': 34, 'width': 219...","{'Entity': 'Bus', 'Door Location': 'NA', 'Vehi..."
1,image000002.jpg,72688,{},1,0,"{'name': 'rect', 'x': 373, 'y': 263, 'width': ...","{'Entity': 'Bus', 'Door Location': 'NA', 'Vehi..."
2,image000003.jpg,82261,{},3,0,"{'name': 'rect', 'x': 74, 'y': 52, 'width': 61...","{'Entity': 'Bus', 'Door Location': 'NA', 'Vehi..."
3,image000003.jpg,82261,{},3,1,"{'name': 'rect', 'x': 465, 'y': 151, 'width': ...","{'Entity': 'Door', 'Door Location': 'Front Doo..."
4,image000003.jpg,82261,{},3,2,"{'name': 'rect', 'x': 638, 'y': 180, 'width': ...","{'Entity': 'Door', 'Door Location': 'Rear Door..."


In [ ]:
def get_img_shape(im):
  img = cv2.imread(imgpath+im)
  return list(img.shape)

In [ ]:
get_img_shape('file104.jpg') 
#Returns [Height,Width,channels]

[266, 474, 3]

In [ ]:
#df1['imgsize'] = df1['filename'].apply(get_img_shape)

In [ ]:
df1.head()

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,image000001.jpg,11432,{},1,0,"{'name': 'rect', 'x': 1, 'y': 34, 'width': 219...","{'Entity': 'Bus', 'Door Location': 'NA', 'Vehi..."
1,image000002.jpg,72688,{},1,0,"{'name': 'rect', 'x': 373, 'y': 263, 'width': ...","{'Entity': 'Bus', 'Door Location': 'NA', 'Vehi..."
2,image000003.jpg,82261,{},3,0,"{'name': 'rect', 'x': 74, 'y': 52, 'width': 61...","{'Entity': 'Bus', 'Door Location': 'NA', 'Vehi..."
3,image000003.jpg,82261,{},3,1,"{'name': 'rect', 'x': 465, 'y': 151, 'width': ...","{'Entity': 'Door', 'Door Location': 'Front Doo..."
4,image000003.jpg,82261,{},3,2,"{'name': 'rect', 'x': 638, 'y': 180, 'width': ...","{'Entity': 'Door', 'Door Location': 'Rear Door..."


In [ ]:
classes = ['busfront','busrear','busside','frontdoor','reardoor','busroute']

In [ ]:
def convert_annotation(annot,size):
  box = np.array([annot['x'], annot['y'], annot['width'], annot['height']],
                                                dtype=np.float32).ravel()
  box[[0, 2]] /= size[1]  # normalize x by width
  box[[1, 3]] /= size[0]  # normalize y by height
  box = [box[0] + box[2] / 2, box[1] + box[3] / 2, box[2],
          box[3]]  # xywh (left-top to center x-y)
  return box

In [ ]:
classes

['busfront', 'busrear', 'busside', 'frontdoor', 'reardoor', 'busroute']

In [ ]:
cd /content/drive/MyDrive/Data/temp/annotations

/content/drive/.shortcut-targets-by-id/1cvx03MF6sl6kQ1ZVtu4SLyCyNCfsSnxC/Data/temp/annotations


In [ ]:
for count, filename in enumerate(os.listdir(imgpath)):
  rows = df1[df1['filename']==filename]
  for index, row in rows.iterrows():
    size = get_img_shape(row['filename'])
    if row['region_count'] == 0:
      continue
    bbox = convert_annotation(row['region_shape_attributes'],size)
    region = row['region_attributes']
    for cat in region.values(): 
      if cat=='Front Pose':
        category_id = 0
      elif cat=='Rear Pose':
        category_id = 1
      elif cat=='Side Pose':
        category_id = 2
      elif cat=='Front Door':
        category_id = 3 
      elif cat=='Rear Door':
        category_id = 4
      elif cat=='Rear Door':
        category_id = 5    
    with open(row['filename'].split('.')[0] + '.txt', 'a') as file:
      file.write('%g %.6f %.6f %.6f %.6f\n' % (category_id, bbox[0],bbox[1],bbox[2],bbox[3]))

In [62]:
annotspath = '/content/drive/MyDrive/Data/temp/annotations/'
dir_list = os.listdir()
with ZipFile('annotations.zip', 'w') as zipObj:
  for file in dir_list:
    zipObj.write(file)